# 21.4.7 Generating Monte Carlo

We are going to generate a small MC sample using a steering file just as if we were preparing an analysis.

In [1]:
import basf2 as b2
main = b2.Path()
# choose to use our own decay file generator or the default's
generate_my_signal_sample = True

We will generate 10 events of on experiment (experiment #0):

In [2]:
main.add_module('EventInfoSetter', evtNumList=[10], expList=[0])

Events will be of the kind $B^0 \bar{B}^0$

In [3]:
# finalstate="mixed" is used to generate B0-B0bar pairs
import generators
generators.add_evtgen_generator(
    path=main,
    finalstate="mixed",
    signaldecfile=b2.find_file("./B0_JpsiKs0_decay.dec") if generate_my_signal_sample else None
)

Welcome to JupyROOT 6.20/04
[WARNING] ignoring decfile: /gpfs/home/belle2/pegueros/workshop/B0_JpsiKs0_decay.dec


Now, we want to simulate the detectors response

In [4]:
import simulation
simulation.add_simulation(
    path=main,
)


**************************************************************
 Geant4 version Name: geant4-10-06-patch-01    (14-February-2020)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************



Now, we want to simulate the L1 trigger

In [5]:
import L1trigger as l1
l1.add_tsim(path=main)

Now, we'll reconstructing the relevant objects, like tracks, electromagnetic clusters, etc

In [6]:
import reconstruction as r
# We set components=None to reconstruct ALL the parts of the experiment
r.add_reconstruction(
    path=main,
    components=None
)

Store the reconstructed objects into an output mDST file for further studies

In [7]:
import mdst
filename= f"my_montecarlo{'_custom' if generate_my_signal_sample else ''}.root"
mdst.add_mdst_output(
    path=main,
    filename=filename
)

Generate, baby!

In [8]:
print(f"Generating {filename}...")
b2.process(path=main)